### Your turn

The above map shows everywhere you can get to from Lexington on a direct flight.  Your job is to:

1. Make a map of all the possible destinations with one transfer. 
2. Make a map of all the possible desitnations with two transfers. 

Make the maps look nice!  Use color coding, vary the size of the features, or be selective about what you display in order to communicate the information effectively.  

Bonus: This is the air travel version of the Kevin Bacon game.  What is the number N, such that you can reach every airport in the world with N or fewer transfers?  

Extra Bonus: Use this very important piece of knowledge to impress your friends at parties!

In [1]:
import folium
import pandas as pd
import numpy as np

In [2]:
# These files use \N as a missing value indicator.  When reading the CSVs, we will tell
# it to use that value as missing or NA.  The double backslash is required because
# otherwise it will interpret \N as a carriage return. 

# Read in the airports data.
airports = pd.read_csv("data/airports.dat", header=None, na_values='\\N')
airports.columns = ["id", "name", "city", "country", "iata", "icao", "latitude", "longitude", "altitude","timezone", "dst", "tz", "type", "source"]

# Read in the airlines data.
airlines = pd.read_csv("data/airlines.dat", header=None, na_values='\\N')
airlines.columns = ["id", "name", "alias", "iata", "icao", "callsign", "country", "active"]

# Read in the routes data.
routes = pd.read_csv("data/routes.dat", header=None, na_values='\\N')
routes.columns = ["airline", "airline_id", "source", "source_id", "dest", "dest_id", "codeshare", "stops", "equipment"]

In [3]:
# one solution

print('There are a total of ' + str(len(airports)) + ' in the world.')

connected_airports = {'LEX'}
for i in range(0,15): 
    dest_airports = set()
    
    for index, row in routes.iterrows(): 
        if row['source'] in connected_airports: 
            dest_airports.add(row['dest'])
            
    # this creates the union of the sets
    connected_airports = connected_airports | dest_airports
    
    print('Within ' + str(i) + ' transfers we can reach a total of ' + str(len(connected_airports)) + ' airports.')


There are a total of 7184 in the world.
Within 0 transfers we can reach a total of 14 airports.
Within 1 transfers we can reach a total of 402 airports.
Within 2 transfers we can reach a total of 1857 airports.
Within 3 transfers we can reach a total of 3056 airports.
Within 4 transfers we can reach a total of 3300 airports.
Within 5 transfers we can reach a total of 3357 airports.
Within 6 transfers we can reach a total of 3373 airports.
Within 7 transfers we can reach a total of 3377 airports.
Within 8 transfers we can reach a total of 3378 airports.
Within 9 transfers we can reach a total of 3378 airports.
Within 10 transfers we can reach a total of 3378 airports.
Within 11 transfers we can reach a total of 3378 airports.
Within 12 transfers we can reach a total of 3378 airports.
Within 13 transfers we can reach a total of 3378 airports.
Within 14 transfers we can reach a total of 3378 airports.


In [4]:
# seems like we max out at 3378 airports.  Let's check to see if that's right. 
# the apply function is a shortcut to avoid looping
# to see what's in there, dump the data to a csv and have a look

airports['connected'] = airports['iata'].apply(lambda x: x in connected_airports)
airports.to_csv('airports.csv')

In [5]:
# it looks like there are a bunch that don't have an airport code, so drop those, and 
# see how many valid aiports are left

airports = airports.dropna(subset=['iata'])
airports.to_csv('airports.csv')
len(airports)

5652

In [6]:
# have a look at the unconnected airports.  Have you heard of any of those?  
# maybe they don't have any commercial flights going there.  
# create a separate set of all airports that have a route going there. 

airports_with_flights = set()

for index, row in routes.iterrows(): 
    airports_with_flights.add(row['source'])
    airports_with_flights.add(row['dest'])
        
print('There are a total of ' + str(len(airports_with_flights)) + ' airports with scheduled flights.')


There are a total of 3425 airports with scheduled flights.


In [7]:
# there are still a few we can't reach.  Drop those without flights, and look at those that are still detached

airports['has_flights'] = airports['iata'].apply(lambda x: x in airports_with_flights)
airports = airports[airports['has_flights']==True]
airports.to_csv('airports.csv')

In [8]:
# these are our leftovers.  Boing Field makes sense.  Not sure why Melbourne is on there.  Don't know the others. 
# you can dig into these more to see what is going on.  Maybe the other end is not a valid airport

airports[airports['connected']==False]

,id,name,city,country,iata,icao,latitude,longitude,altitude,timezone,dst,tz,type,source,connected,has_flights
1560,1602,Ovda International Airport,Ovda,Israel,VDA,LLOV,29.940300,34.935799,1492,2.0,E,Asia/Jerusalem,airport,OurAirports,False,True
1928,1998,Koné Airport,Kone,New Caledonia,KNQ,NWWD,-21.054300,164.837006,23,11.0,U,Pacific/Noumea,airport,OurAirports,False,True
1929,1999,Koumac Airport,Koumac,New Caledonia,KOC,NWWK,-20.546301,164.255997,42,11.0,U,Pacific/Noumea,airport,OurAirports,False,True
1930,2000,Lifou Airport,Lifou,New Caledonia,LIF,NWWL,-20.774799,167.240005,92,11.0,U,Pacific/Noumea,airport,OurAirports,False,True
1931,2001,Nouméa Magenta Airport,Noumea,New Caledonia,GEA,NWWM,-22.258301,166.473007,10,11.0,U,Pacific/Noumea,airport,OurAirports,False,True
1932,2002,Maré Airport,Mare,New Caledonia,MEE,NWWR,-21.481701,168.037994,141,11.0,U,Pacific/Noumea,airport,OurAirports,False,True
1933,2003,Touho Airport,Touho,New Caledonia,TOU,NWWU,-20.790001,165.259003,10,11.0,U,Pacific/Noumea,airport,OurAirports,False,True
1934,2004,Ouvéa Airport,Ouvea,New Caledonia,UVE,NWWV,-20.640600,166.572998,23,11.0,U,Pacific/Noumea,airport,OurAirports,False,True
3129,3334,Melbourne Essendon Airport,Melbourne,Australia,MEB,YMEN,-37.728100,144.901993,282,10.0,O,Australia/Hobart,airport,OurAirports,False,True
3509,3726,Boeing Field King County International Airport,Seattle,United States,BFI,KBFI,47.529999,-122.302002,21,-8.0,A,America/Los_Angeles,airport,OurAirports,False,True
